In [ ]:
from code_chunker import build_chunks
from data_processing import get_dataset, tokenize_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import tqdm

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('qwen25_coder_1_5b_instruct')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('qwen25_coder_1_5b_instruct')

_, _, test = get_dataset()
test_dataset = tokenize_dataset(test, tokenizer)

In [ ]:
def get_prediction(prompt, tokenizer, model):
    messages = [
        {"role": "system", "content": "Analyze the following C++ code and classify its vulnerability."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
def predict_code_chunks(root, tokenizer, model):
    chunks = build_chunks(root, tokenizer.encode, max_tokens=1024, overlap=100)
    results = []
    for chunk in chunks:
        response = get_prediction(chunk, tokenizer, model)
        results.append({
            "file": chunk["file"],
            "chunk": chunk["chunk"],
            "code": chunk["code"],
            "response": response
        })
    return results